# 🔄 Workflow Concepts - Microsoft.Agents.AI.Workflows

This notebook explains the key concepts of building AI workflows using the Microsoft Agent Framework.

> 📝 **Hands-on Exercises**: After reviewing these concepts, complete the exercises in [EXERCISES.md](EXERCISES.md).

## Table of Contents

1. [What is a Workflow?](#1-what-is-a-workflow)
2. [Core Building Blocks](#2-core-building-blocks)
3. [Sequential Workflow](#3-sequential-workflow)
4. [Concurrent Workflow](#4-concurrent-workflow)
5. [Human-in-the-Loop Workflow](#5-human-in-the-loop-workflow)
6. [Workflow Events](#6-workflow-events)
7. [Best Practices](#7-best-practices)

## 1. What is a Workflow?

A **Workflow** in Microsoft.Agents.AI is a directed graph of **Executors** connected by **Edges**. Data flows through the workflow, being processed by each executor in sequence or in parallel.

```
┌───────────┐     ┌───────────┐     ┌───────────┐
│ Executor  │────▶│ Executor  │────▶│ Executor  │
│    A      │     │    B      │     │    C      │
└───────────┘     └───────────┘     └───────────┘
     │                                    │
     └─────────── Edge ───────────────────┘
```

### Key Benefits

| Benefit | Description |
|---------|-------------|
| **Composability** | Build complex workflows from simple, reusable components |
| **AI Integration** | Seamlessly integrate AI agents into the workflow |
| **Human Oversight** | Pause workflows to get human input or approval |
| **Streaming** | Process events as they occur, not just at completion |

## Setup: Install Required NuGet Packages

Run the following cell to install the required packages for Workflows and Azure OpenAI integration.

In [ ]:
// Install NuGet packages for Workflows
#r "nuget: Microsoft.Agents.AI.Workflows, 1.0.0-preview.25171.1"
#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.25171.1"
#r "nuget: Microsoft.Extensions.AI.OpenAI, 9.7.0-preview.1.25356.2"
#r "nuget: Azure.AI.OpenAI, 2.3.0-beta.1"
#r "nuget: Azure.Identity, 1.13.2"
#r "nuget: Microsoft.Extensions.Configuration.Json, 10.0.0-preview.4.25258.110"
#r "nuget: Microsoft.Extensions.Configuration.EnvironmentVariables, 10.0.0-preview.4.25258.110"

using System;
using System.IO;
using System.Text.Json;
using System.Threading;
using System.Threading.Tasks;
using System.Collections.Generic;
using Microsoft.Extensions.Configuration;
using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using Microsoft.Agents.AI.Workflows;
using Azure.AI.OpenAI;
using Azure.Identity;

Console.WriteLine("✅ Packages loaded successfully!");

## Configuration Setup

Configure your Azure OpenAI credentials. You can either:
1. Set them directly in the cell below, or
2. Use the `appsettings.Local.json` file in the `dotnet` folder

In [ ]:
// Helper function to find the dotnet folder by traversing up the directory tree
string FindConfigPath(string startPath)
{
    var currentDir = new DirectoryInfo(startPath);

    // Traverse up to find the 'dotnet' folder
    while (currentDir != null)
    {
        if (currentDir.Name.Equals("dotnet", StringComparison.OrdinalIgnoreCase))
        {
            return currentDir.FullName;
        }
        currentDir = currentDir.Parent;
    }

    // Fallback to start path if dotnet folder not found
    return startPath;
}

// Load configuration from appsettings.Local.json in the dotnet folder
var configPath = FindConfigPath(Directory.GetCurrentDirectory());
Console.WriteLine($"📁 Config path: {configPath}");

var configuration = new ConfigurationBuilder()
    .SetBasePath(configPath)
    .AddJsonFile("appsettings.Local.json", optional: true)
    .AddEnvironmentVariables()
    .Build();

// Get Azure OpenAI settings - support both flat and nested config keys
var endpoint = configuration["AZURE_OPENAI_ENDPOINT"] 
    ?? configuration["AzureOpenAI:Endpoint"]
    ?? configuration["AZURE_AI_PROJECT_ENDPOINT"];
    
var apiKey = configuration["AZURE_OPENAI_API_KEY"] 
    ?? configuration["AzureOpenAI:ApiKey"];
    
var deploymentName = configuration["AZURE_OPENAI_DEPLOYMENT_NAME"] 
    ?? configuration["AzureOpenAI:DeploymentName"]
    ?? configuration["AZURE_AI_MODEL_DEPLOYMENT_NAME"]
    ?? "gpt-4o-mini";

// Get Service Principal credentials
var tenantId = configuration["AZURE_TENANT_ID"] ?? configuration["AzureOpenAI:TenantId"];
var clientId = configuration["AZURE_CLIENT_ID"] ?? configuration["AzureOpenAI:ClientId"];
var clientSecret = configuration["AZURE_CLIENT_SECRET"] ?? configuration["AzureOpenAI:ClientSecret"];

if (string.IsNullOrEmpty(endpoint))
{
    Console.WriteLine("⚠️ Azure OpenAI endpoint not configured");
    Console.WriteLine("   Set AZURE_OPENAI_ENDPOINT in appsettings.Local.json or environment");
}
else
{
    Console.WriteLine($"✅ Configuration loaded");
    Console.WriteLine($"   Endpoint: {endpoint}");
    Console.WriteLine($"   Deployment: {deploymentName}");
    
    if (!string.IsNullOrEmpty(apiKey))
        Console.WriteLine($"   Auth: API Key");
    else if (!string.IsNullOrEmpty(tenantId) && !string.IsNullOrEmpty(clientId) && !string.IsNullOrEmpty(clientSecret))
        Console.WriteLine($"   Auth: Service Principal");
    else
        Console.WriteLine($"   Auth: Azure CLI / DefaultAzureCredential");
}

In [ ]:
AzureOpenAIClient aoaiClient;

if (string.IsNullOrEmpty(endpoint))
{
    throw new InvalidOperationException("Azure OpenAI endpoint is not configured. Set AZURE_OPENAI_ENDPOINT in appsettings.Local.json or environment.");
}

if (!string.IsNullOrWhiteSpace(apiKey))
{
    // API Key authentication
    aoaiClient = new AzureOpenAIClient(new Uri(endpoint), new System.ClientModel.ApiKeyCredential(apiKey));
    Console.WriteLine("✅ Created Azure OpenAI client with API Key authentication");
}
else if (!string.IsNullOrWhiteSpace(tenantId) && !string.IsNullOrWhiteSpace(clientId) && !string.IsNullOrWhiteSpace(clientSecret))
{
    // Service Principal authentication
    var credential = new ClientSecretCredential(tenantId, clientId, clientSecret);
    aoaiClient = new AzureOpenAIClient(new Uri(endpoint), credential);
    Console.WriteLine("✅ Created Azure OpenAI client with Service Principal authentication");
}
else
{
    // Fallback to DefaultAzureCredential
    aoaiClient = new AzureOpenAIClient(new Uri(endpoint), new DefaultAzureCredential());
    Console.WriteLine("✅ Created Azure OpenAI client with DefaultAzureCredential");
}

var chatClient = aoaiClient.GetChatClient(deploymentName).AsIChatClient();
Console.WriteLine($"✅ Chat client created for deployment: {deploymentName}");

## Define the SupportTicket Model

Define the SupportTicket record that will be used throughout the workflow examples.

In [ ]:
// Define the SupportTicket model
public enum TicketPriority { Low, Medium, High, Critical }

public record SupportTicket(
    string TicketId,
    string CustomerId,
    string CustomerName,
    string Subject,
    string Description,
    TicketPriority Priority = TicketPriority.Medium
);

// Create a sample ticket for testing
var sampleTicket = new SupportTicket(
    TicketId: "TKT-12345",
    CustomerId: "CUST-001",
    CustomerName: "John Smith",
    Subject: "Cannot login to my account",
    Description: "I've been trying to login for the past hour but keep getting an 'invalid credentials' error even though I just reset my password.",
    Priority: TicketPriority.High
);

Console.WriteLine($"✅ Sample ticket created: {sampleTicket.TicketId} - {sampleTicket.Subject}");

## 2. Core Building Blocks

### 2.1 Executors

An **Executor** is a unit of work in a workflow. It receives input, processes it, and optionally sends output to the next executor.

#### Class-Based Executor (Single Input Type)

```csharp
using Microsoft.Agents.AI.Workflows;
using Microsoft.Extensions.AI;

// Executor that receives a SupportTicket and processes it
internal sealed class TicketIntakeExecutor() : Executor<SupportTicket>("TicketIntake")
{
    public override async ValueTask HandleAsync(
        SupportTicket ticket, 
        IWorkflowContext context, 
        CancellationToken cancellationToken = default)
    {
        // Validate the ticket
        if (string.IsNullOrWhiteSpace(ticket.Subject))
            throw new ArgumentException("Subject is required");

        // Format the ticket for the AI agent
        var ticketText = $"Ticket ID: {ticket.TicketId}\nSubject: {ticket.Subject}";

        // Send as a user message to the next executor (AI agent)
        await context.SendMessageAsync(new ChatMessage(ChatRole.User, ticketText), cancellationToken);
        
        // Trigger the AI agent to process the message
        await context.SendMessageAsync(new TurnToken(emitEvents: true), cancellationToken);
    }
}
```

#### Executor Processing AI Agent Output

```csharp
// Executor that receives output from an AI agent (List<ChatMessage>)
internal sealed class CategorizationBridgeExecutor() : Executor<List<ChatMessage>>("CategorizationBridge")
{
    public override async ValueTask HandleAsync(
        List<ChatMessage> messages, 
        IWorkflowContext context, 
        CancellationToken cancellationToken = default)
    {
        // Get the AI's response
        var categorizationResult = messages.LastOrDefault()?.Text ?? "Unknown";

        // Prepare for next AI agent
        var prompt = $"Based on: {categorizationResult}\nGenerate a response.";
        
        await context.SendMessageAsync(new ChatMessage(ChatRole.User, prompt), cancellationToken);
        await context.SendMessageAsync(new TurnToken(emitEvents: true), cancellationToken);
    }
}
```

#### Executor That Yields Output

```csharp
// Executor that produces the final workflow output
internal sealed class ResponseBridgeExecutor() : Executor<List<ChatMessage>>("ResponseBridge")
{
    public override async ValueTask HandleAsync(
        List<ChatMessage> messages, 
        IWorkflowContext context, 
        CancellationToken cancellationToken = default)
    {
        var response = messages.LastOrDefault()?.Text ?? "Unable to generate response.";
        
        // Yield the final output
        await context.YieldOutputAsync(response, cancellationToken);
    }
}
```

### 2.2 AI Agents (ChatClientAgent)

**ChatClientAgent** integrates AI models (like Azure OpenAI) into workflows.

```csharp
using Microsoft.Agents.AI;
using Microsoft.Extensions.AI;

// Create an AI agent with specific instructions
ChatClientAgent categorizationAgent = new(
    chatClient,                    // IChatClient from Azure OpenAI
    name: "CategorizationAgent",
    instructions: """
        You are a customer support ticket categorization specialist.
        Analyze the incoming support ticket and categorize it into:
        - BILLING: Payment issues, invoices, subscription
        - TECHNICAL: Software bugs, errors, how-to questions
        - GENERAL: Account inquiries, feedback
        
        Respond with JSON: {"category": "CATEGORY", "priority": "HIGH|MEDIUM|LOW"}
        """
);
```

### Key Concepts for AI Agents

| Concept | Description |
|---------|-------------|
| **ChatMessage** | User or assistant message sent to/from the AI |
| **TurnToken** | Triggers the AI agent to process accumulated messages |
| **emitEvents** | When true, the agent emits completion events |
| **List<ChatMessage>** | Output type from AI agents containing all messages |

### 2.3 Workflow Builder

The **WorkflowBuilder** connects executors with edges to create a workflow.

```csharp
using Microsoft.Agents.AI.Workflows;

var workflow = new WorkflowBuilder(entryExecutor)  // First executor
    .AddEdge(executorA, executorB)                  // Connect A → B
    .AddEdge(executorB, executorC)                  // Connect B → C
    .WithOutputFrom(executorC)                      // Define output source
    .Build();                                       // Create the workflow
```

### Edge Types

| Edge Type | Method | Description |
|-----------|--------|-------------|
| **Direct** | `AddEdge(a, b)` | Simple A → B connection |
| **Fan-Out** | `AddFanOutEdge(a, [b, c, d])` | A → [B, C, D] (parallel) |
| **Fan-In** | `AddFanInEdge([a, b, c], d)` | [A, B, C] → D (aggregate) |

### 2.4 Workflow Execution

Execute workflows and process events using streaming:

```csharp
using Microsoft.Agents.AI.Workflows;

// Start streaming execution
await using StreamingRun run = await InProcessExecution.StreamAsync(workflow, inputData);

// Process events as they occur
await foreach (WorkflowEvent evt in run.WatchStreamAsync())
{
    switch (evt)
    {
        case ExecutorCompletedEvent completed:
            Console.WriteLine($"✓ [{completed.ExecutorId}] completed");
            if (completed.Data is string data)
                Console.WriteLine($"   Output: {data}");
            break;
            
        case WorkflowOutputEvent output:
            Console.WriteLine($"Output: {output.Data}");
            break;
            
        case RequestInfoEvent request:
            // Handle external input request (human-in-the-loop)
            var reviewRequest = request.Request.DataAs<SupervisorReviewRequest>();
            var decision = GetHumanDecision(reviewRequest);
            var response = request.Request.CreateResponse(decision);
            await run.SendResponseAsync(response);
            break;
    }
}
```

## 3. Sequential Workflow

A **Sequential Workflow** processes data through a linear pipeline where each step depends on the previous one.

### Architecture

```
┌─────────────┐    ┌──────────────────┐    ┌─────────────────────┐    ┌──────────────┐
│   Ticket    │───▶│  Categorization  │───▶│  Categorization     │───▶│   Response   │
│   Intake    │    │   AI Agent       │    │  Bridge Executor    │    │   AI Agent   │
└─────────────┘    └──────────────────┘    └─────────────────────┘    └──────────────┘
                                                                              │
                                                                              ▼
                                                                    ┌──────────────┐
                                                                    │   Response   │
                                                                    │    Bridge    │
                                                                    └──────────────┘
```

### Implementation from WorkflowLab

```csharp
// 1. Create executors
var ticketIntake = new TicketIntakeExecutor();
var categorizationBridge = new CategorizationBridgeExecutor();
var responseBridge = new ResponseBridgeExecutor();

// 2. Create AI agents
ChatClientAgent categorizationAgent = new(
    chatClient,
    name: "CategorizationAgent",
    instructions: """
        You are a customer support ticket categorization specialist.
        Analyze the incoming support ticket and categorize it into:
        - BILLING, TECHNICAL, or GENERAL
        Respond with JSON: {"category": "CATEGORY", "priority": "HIGH|MEDIUM|LOW", "summary": "brief summary"}
        """
);

ChatClientAgent responseAgent = new(
    chatClient,
    name: "ResponseAgent",
    instructions: """
        You are a friendly customer support representative.
        Generate a helpful, empathetic response to the customer.
        Keep it concise (3-4 sentences).
        """
);

// 3. Build sequential workflow
var workflow = new WorkflowBuilder(ticketIntake)
    .AddEdge(ticketIntake, categorizationAgent)          // Intake → AI Categorize
    .AddEdge(categorizationAgent, categorizationBridge)  // AI → Bridge
    .AddEdge(categorizationBridge, responseAgent)        // Bridge → AI Respond
    .AddEdge(responseAgent, responseBridge)              // AI → Output
    .WithOutputFrom(responseBridge)
    .Build();
```

### Sending Messages to AI Agents

```csharp
public override async ValueTask HandleAsync(SupportTicket ticket, IWorkflowContext context, ...)
{
    // 1. Format the prompt
    var ticketText = $"""
        Ticket ID: {ticket.TicketId}
        Customer: {ticket.CustomerName}
        Subject: {ticket.Subject}
        Description: {ticket.Description}
        """;
    
    // 2. Send as a user message
    await context.SendMessageAsync(new ChatMessage(ChatRole.User, ticketText), cancellationToken);
    
    // 3. Send TurnToken to trigger the AI agent
    await context.SendMessageAsync(new TurnToken(emitEvents: true), cancellationToken);
}
```

### Use Cases

- **Document Processing**: Parse → Extract → Summarize → Store
- **Order Processing**: Validate → Calculate → Confirm → Fulfill
- **Content Pipeline**: Draft → Review → Edit → Publish
- **Support Tickets**: Intake → Categorize → Respond

## 4. Concurrent Workflow

A **Concurrent Workflow** distributes work to multiple executors simultaneously and aggregates results.

### Architecture

```
                         ┌──────────────────┐
                    ┌───▶│  Billing Expert  │────┐
┌─────────────┐     │    └──────────────────┘    │    ┌─────────────────┐
│   Start     │─────│                            │───▶│   Aggregation   │
│  Executor   │     │    ┌──────────────────┐    │    │    Executor     │
└─────────────┘     └───▶│ Technical Expert │────┘    └─────────────────┘
                         └──────────────────┘
        │                                                     │
        └──────── Fan-Out ──────── Fan-In ────────────────────┘
```

### Implementation from WorkflowLab

```csharp
// 1. Create multiple AI agents
ChatClientAgent billingExpert = new(
    chatClient,
    name: "BillingExpert",
    instructions: """
        You are an expert in billing and subscription matters.
        Analyze the customer's question from a billing perspective.
        If not billing-related, briefly acknowledge and defer.
        Keep responses concise (2-3 sentences).
        """
);

ChatClientAgent technicalExpert = new(
    chatClient,
    name: "TechnicalExpert",
    instructions: """
        You are an expert in technical support and troubleshooting.
        Analyze the customer's question from a technical perspective.
        If not technical, briefly acknowledge and defer.
        Keep responses concise (2-3 sentences).
        """
);

// 2. Create start and aggregation executors
var startExecutor = new ConcurrentStartExecutor();
var aggregationExecutor = new ConcurrentAggregationExecutor();

// 3. Build concurrent workflow with fan-out/fan-in
var workflow = new WorkflowBuilder(startExecutor)
    .AddFanOutEdge(startExecutor, targets: [billingExpert, technicalExpert])
    .AddFanInEdge(sources: [billingExpert, technicalExpert], aggregationExecutor)
    .WithOutputFrom(aggregationExecutor)
    .Build();
```

### Start Executor (Broadcasting)

```csharp
internal sealed class ConcurrentStartExecutor() : Executor<string>("ConcurrentStart")
{
    public override async ValueTask HandleAsync(string message, IWorkflowContext context, ...)
    {
        // Broadcast the message to all connected agents
        await context.SendMessageAsync(new ChatMessage(ChatRole.User, message), cancellationToken);

        // Broadcast the turn token to kick off all agents
        await context.SendMessageAsync(new TurnToken(emitEvents: true), cancellationToken);
    }
}
```

### Aggregation Executor (Collecting Results)

```csharp
internal sealed class ConcurrentAggregationExecutor() : Executor<List<ChatMessage>>("ConcurrentAggregation")
{
    private readonly List<ChatMessage> _messages = [];

    public override async ValueTask HandleAsync(List<ChatMessage> messages, IWorkflowContext context, ...)
    {
        _messages.AddRange(messages);

        // Wait for responses from all agents (2 in this demo)
        if (_messages.Count >= 2)
        {
            var formattedMessages = string.Join(
                Environment.NewLine + Environment.NewLine,
                _messages.Select(m => $"[{m.AuthorName}]: {m.Text}")
            );

            await context.YieldOutputAsync(formattedMessages, cancellationToken);
        }
    }
}
```

### Use Cases

- **Multi-Expert Analysis**: Get perspectives from multiple specialists
- **Parallel Processing**: Process multiple items simultaneously
- **Consensus Building**: Aggregate opinions from multiple AI agents
- **Redundancy**: Get multiple answers for verification

## 5. Human-in-the-Loop Workflow

A **Human-in-the-Loop Workflow** pauses execution to get human input, approval, or oversight.

### Architecture

```
┌─────────────┐    ┌──────────────┐    ┌──────────────────┐    ┌──────────────┐
│   Ticket    │───▶│   AI Draft   │───▶│   RequestPort    │───▶│   Finalize   │
│   Intake    │    │    Agent     │    │ (Human Review)   │    │   Response   │
└─────────────┘    └──────────────┘    └──────────────────┘    └──────────────┘
                                               │
                                    ┌──────────┴───────────┐
                                    │     Supervisor       │
                                    │   ┌─────────────┐    │
                                    │   │  Approve    │    │
                                    │   │  Edit       │    │
                                    │   │  Escalate   │    │
                                    │   └─────────────┘    │
                                    └──────────────────────┘
```

### Key Component: RequestPort

```csharp
// Define request and response types
public sealed record SupervisorReviewRequest(
    string TicketId,
    string Category,
    string Priority,
    string DraftResponse
);

public sealed record SupervisorDecision(
    ReviewAction Action,      // Approve, Edit, Escalate
    string? ModifiedResponse,
    string? Notes
);

public enum ReviewAction { Approve, Edit, Escalate }

// Create the RequestPort
RequestPort reviewPort = RequestPort.Create<SupervisorReviewRequest, SupervisorDecision>("SupervisorReview");
```

### Building the Workflow

```csharp
// Create AI agent for drafting
ChatClientAgent draftAgent = new(
    chatClient,
    name: "DraftAgent",
    instructions: """
        You are an experienced customer support specialist.
        Analyze the ticket and draft a professional, empathetic response.
        For refund requests, explain the 14-day money-back guarantee.
        Keep your response to 3-5 sentences.
        """
);

// Create executors
var ticketIntake = new HumanInTheLoopTicketIntakeExecutor();
var draftBridge = new DraftBridgeExecutor();
RequestPort supervisorReviewPort = RequestPort.Create<SupervisorReviewRequest, SupervisorDecision>("SupervisorReview");
var finalizeExecutor = new FinalizeExecutor();

// Build workflow
var workflow = new WorkflowBuilder(ticketIntake)
    .AddEdge(ticketIntake, draftAgent)               // Intake → AI Draft
    .AddEdge(draftAgent, draftBridge)                // AI → Prepare Review
    .AddEdge(draftBridge, supervisorReviewPort)      // Bridge → RequestPort (PAUSE)
    .AddEdge(supervisorReviewPort, finalizeExecutor) // Response → Finalize
    .WithOutputFrom(finalizeExecutor)
    .Build();
```

### Sending to RequestPort

```csharp
// In DraftBridgeExecutor
public override async ValueTask HandleAsync(List<ChatMessage> messages, IWorkflowContext context, ...)
{
    var draftResponse = messages.LastOrDefault()?.Text ?? "Unable to generate.";
    var ticket = HumanInTheLoopTicketIntakeExecutor.CurrentTicket!;

    // Create review request
    var reviewRequest = new SupervisorReviewRequest(
        TicketId: ticket.TicketId,
        Category: "REFUND",
        Priority: ticket.Priority.ToString(),
        DraftResponse: draftResponse
    );

    // Send to RequestPort - workflow will pause here
    await context.SendMessageAsync(reviewRequest, cancellationToken);
}
```

### Handling the Human Input

```csharp
await foreach (WorkflowEvent evt in run.WatchStreamAsync())
{
    switch (evt)
    {
        case RequestInfoEvent requestEvt:
            // Workflow is paused, waiting for human input
            var request = requestEvt.Request.DataAs<SupervisorReviewRequest>();
            
            // Display to supervisor
            Console.WriteLine($"Review needed for: {request.TicketId}");
            Console.WriteLine($"Draft: {request.DraftResponse}");
            Console.WriteLine("[1] Approve [2] Edit [3] Escalate");
            
            // Get human decision
            var decision = new SupervisorDecision(
                Action: ReviewAction.Approve,
                ModifiedResponse: null,
                Notes: "Approved as-is"
            );
            
            // Resume workflow with the response
            var response = requestEvt.Request.CreateResponse(decision);
            await run.SendResponseAsync(response);
            break;
            
        case WorkflowOutputEvent output:
            Console.WriteLine($"Final: {output.Data}");
            break;
    }
}
```

### Use Cases

- **Content Approval**: AI drafts, human approves before publishing
- **Financial Decisions**: AI recommends, human authorizes large transactions
- **Customer Escalation**: AI handles routine, human handles exceptions
- **Quality Control**: AI processes, human spot-checks
- **Compliance**: AI suggests, human verifies compliance

## 6. Workflow Events

Workflows emit events that you can process in real-time:

| Event Type | Description | When Emitted |
|------------|-------------|---------------|
| `ExecutorCompletedEvent` | An executor finished processing | After each executor completes |
| `WorkflowOutputEvent` | Workflow produced final output | When `YieldOutputAsync` is called |
| `RequestInfoEvent` | Workflow needs external input | When a `RequestPort` is reached |

### Example: Processing All Events

```csharp
await using StreamingRun run = await InProcessExecution.StreamAsync(workflow, input);

await foreach (WorkflowEvent evt in run.WatchStreamAsync())
{
    switch (evt)
    {
        case ExecutorCompletedEvent completed:
            Console.WriteLine($"✓ [{completed.ExecutorId}] completed");
            if (completed.Data is string data && !string.IsNullOrWhiteSpace(data))
                Console.WriteLine($"   Output: {data}");
            break;

        case WorkflowOutputEvent output:
            Console.WriteLine($"📤 Final Output: {output.Data}");
            break;

        case RequestInfoEvent request:
            Console.WriteLine($"⏸️ Waiting for input: {request.Request.PortId}");
            // Get human input and respond...
            var response = request.Request.CreateResponse(decision);
            await run.SendResponseAsync(response);
            break;
    }
}
```

## 7. Best Practices

### 7.1 Executor Design

- **Single Responsibility**: Each executor should do one thing well
- **Stateless When Possible**: Avoid storing state in executors (use context or external storage)
- **Clear Naming**: Use descriptive names that indicate the executor's purpose
- **Bridge Executors**: Use bridge executors between AI agents to transform data

### 7.2 AI Agent Instructions

```csharp
// ✅ Good: Specific, structured output
ChatClientAgent agent = new(chatClient,
    name: "Categorizer",
    instructions: """
        Categorize tickets into: BILLING, TECHNICAL, GENERAL
        Output JSON: {"category": "CATEGORY", "confidence": 0.0-1.0}
        Be concise. Only output the JSON.
        """);

// ❌ Bad: Vague, unstructured
ChatClientAgent agent = new(chatClient,
    name: "Helper",
    instructions: "Help with customer support");
```

### 7.3 Message Flow Pattern

```csharp
// Standard pattern for sending to AI agents:
// 1. Send ChatMessage with prompt
await context.SendMessageAsync(new ChatMessage(ChatRole.User, prompt), ct);

// 2. Send TurnToken to trigger processing
await context.SendMessageAsync(new TurnToken(emitEvents: true), ct);

// Standard pattern for yielding output:
await context.YieldOutputAsync(result, ct);
```

### 7.4 Error Handling

```csharp
try
{
    await using var run = await InProcessExecution.StreamAsync(workflow, input);
    await foreach (var evt in run.WatchStreamAsync())
    {
        // Process events...
    }
}
catch (Exception ex)
{
    Console.WriteLine($"Workflow error: {ex.Message}");
    // Log, retry, or escalate
}
```

### 7.5 Configuration Management

```csharp
// Use IConfiguration for flexible config
var config = new ConfigurationBuilder()
    .AddJsonFile("appsettings.json", optional: true)
    .AddJsonFile("appsettings.Development.json", optional: true)
    .AddEnvironmentVariables()
    .Build();

// Support multiple auth methods
// 1. API Key
// 2. Service Principal (TenantId, ClientId, ClientSecret)
// 3. Managed Identity / DefaultAzureCredential
```

## Summary

| Pattern | When to Use | Key Classes |
|---------|-------------|-------------|
| **Sequential** | Linear processing pipeline | `AddEdge()`, `Executor<T>` |
| **Concurrent** | Parallel processing, multi-expert | `AddFanOutEdge()`, `AddFanInEdge()` |
| **Human-in-the-Loop** | Approval, oversight, exceptions | `RequestPort`, `RequestInfoEvent` |

### Quick Reference

```csharp
// Create AI agent
ChatClientAgent agent = new(chatClient, name: "Agent", instructions: "...");

// Build workflow
var workflow = new WorkflowBuilder(entryExecutor)
    .AddEdge(a, b)                          // Sequential
    .AddFanOutEdge(a, [b, c])               // Parallel
    .AddFanInEdge([b, c], d)                // Aggregate
    .WithOutputFrom(outputExecutor)
    .Build();

// Execute
await using var run = await InProcessExecution.StreamAsync(workflow, input);
await foreach (var evt in run.WatchStreamAsync()) { /* handle events */ }

// Send to AI agent (in executor)
await context.SendMessageAsync(new ChatMessage(ChatRole.User, prompt), ct);
await context.SendMessageAsync(new TurnToken(emitEvents: true), ct);

// Yield output
await context.YieldOutputAsync(result, ct);

// Create RequestPort for human-in-the-loop
RequestPort port = RequestPort.Create<TRequest, TResponse>("PortName");
```